# Check Observations for the night February 15th 2022 


https://harvard.zoom.us/rec/play/N7umVF8Rk3vWvW_O3jkaTinofB2WxW3QKz0fVai_K-qvAYLt37uMrvskOlnBiMEjduSeQVzWT7_DDYzc.IY7Pqjef82YlWmZE?continueMode=true&_x_zm_rtaid=BjQYrDaETYeYRUMw8NQgLw.1613559822570.c813d210d38e378b719cb4048493df5b&_x_zm_rhtaid=205


- work with **Weakly_2021_44** : this is because I have holoviw with this version


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/02/25
- update : 22/02/26


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
import os,sys
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

In [ ]:
# Bokeh for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize
from holoviews.operation.datashader import datashade, dynspread
from holoviews.plotting.util import process_cmap


import datashader as dsh

In [ ]:
#! pip install holoview

In [ ]:
# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
#hv.extension('bokeh')
hv.extension('bokeh', 'matplotlib')

# Display bokeh plots inline in the notebook
output_notebook()

# Check executable

In [ ]:
print(sys.executable)
print(sys.version)
#print(sys.version_info)

# Configuration

## Selection Flag

In [ ]:
FLAG_ROTATE_IMG = True
FLAG_TRANSFORM = True

## Range of values

In [ ]:
VMIN = -10
VMAX = 60

## transformation of the image

https://docs.astropy.org/en/stable/visualization/index.html

In [ ]:
#transform = AsinhStretch() + PercentileInterval(99.)
transform = ZScaleInterval(contrast=0.5)
#transform = PercentileInterval(99.)

## Histograms

In [ ]:
NBINS_HISTO = 200

## Holoview Elements config

### Histogram config

In [ ]:
HV_HISTO_SINGLE_WIDTH  = 400
HV_HISTO_SINGLE_HEIGHT = 350
HV_HISTO_MULTI_WIDTH  = 350
HV_HISTO_MULTI_HEIGHT = 300
HV_HISTO_MULTI_COLS   = 4

### Image config

In [ ]:
HV_IMAGE_SINGLE_WIDTH  = 800
HV_IMAGE_SINGLE_HEIGHT = 800
HV_IMAGE_SINGLE_FRAME_WIDTH = 800
HV_IMAGE_MULTI_WIDTH  = 300
HV_IMAGE_MULTI_HEIGHT = 300
HV_IMAGE_MULTI_FRAME_WIDTH = 300
HV_IMAGE_MULTI_COLS   = 4

# Path of the image

In [ ]:
MYDATE="2022-02-15"

In [ ]:
path_postISRCCD="/sps/lsst/groups/auxtel/softs/shared/auxteldm/rerun/dagoret_testfordispersers2022_03/postISRCCD/" + MYDATE

In [ ]:
list_of_files_postisrccd=os.listdir(path_postISRCCD)

In [ ]:
list_of_files_postisrccd = sorted(list_of_files_postisrccd)
list_of_files_postisrccd

# Select the image

In [ ]:
index=6

In [ ]:
filename_postisrccd = list_of_files_postisrccd[index]
fullfilename_postisrccd=os.path.join(path_postISRCCD,filename_postisrccd)
hdul_postisrccd = fits.open(fullfilename_postisrccd)

In [ ]:
filename_name = filename_postisrccd 

In [ ]:
hdul_postisrccd.info()

# Process the image

In [ ]:
image=hdul_postisrccd[1].data

In [ ]:
if FLAG_TRANSFORM: 
    scaledImage = transform(image)
else:
    scaledImage = image

In [ ]:
if FLAG_ROTATE_IMG:
    rotscaledImage=np.rot90(scaledImage)
else:
    rotscaledImage = scaledImage

In [ ]:
#from holoviews import streams
from holoviews.streams import Stream, param
listing = ', '.join(sorted([str(s.name) for s in param.descendents(streams.LinkedStream)]))
print('The linked stream classes supported by HoloViews are:\n\n{listing}'.format(listing=listing))

In [ ]:
TOOLTIPS = [
    ('name', "$name"),
    ('index', "$index"),
    ('pattern', '@pattern'),
    ("x", "$x"),
    ("y", "$y"),
    ("value", "@image"),
    ('squared', '@squared')
]

TOOLTIPS = [
    ("(x,y)", "($x, $y)"),
]

hover = HoverTool(description='Custom Tooltip', tooltips=[('x', '@x'), ('y', '@y')])


# Custom hover tool for the source detections
myhover = HoverTool(
    tooltips=[
        ( 'x', '@x{0.2f}'),
        ( 'y', '@y{0.2f}'),
    ],
    formatters={
        'x' : 'printf',
        'y' : 'printf',
    },
    
)

In [ ]:
# Define some default plot options for the Image
img_opts = dict(
                #height=600, width=700, 
                xaxis="bottom", 
                padding = 0.0, fontsize={'title': '8pt'},
                colorbar=True, toolbar='right', show_grid=True,
                aspect='equal',
                frame_width=HV_IMAGE_SINGLE_FRAME_WIDTH
               )   

In [ ]:
# Create the Image element.
bounds_img=(0,0,rotscaledImage.shape[1],rotscaledImage.shape[0])
img = hv.Image(rotscaledImage, bounds=bounds_img,
               kdims=['x', 'y']).opts(
    cmap = "Greys_r",  xlabel = 'x', ylabel ='y',
#    clim=(VMIN,VMAX),
    title = filename_name,
    **img_opts)

In [ ]:
rasterize(img)